<a href="https://colab.research.google.com/github/itimes-digital/deep-learning-estudo/blob/main/gan_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/bstriner/keras_adversarial.git
!cd keras_adversarial
!python setup.py reinstall

fatal: destination path 'keras_adversarial' already exists and is not an empty directory.
python3: can't open file 'setup.py': [Errno 2] No such file or directory


In [1]:
import keras
keras.__version__

'2.4.3'

In [2]:
pip install keras==2.1.2

     |████████████████████████████████| 307kB 12.2MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.1.2 which is incompatible.
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [1]:
!pip install tensorflow==1.2

     |████████████████████████████████| 35.0MB 122kB/s 
     |████████████████████████████████| 890kB 30.3MB/s 
     |████████████████████████████████| 245kB 47.9MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=20c6dd0231fc7a54801760420f1e732e7c6fb205cb4a2571ac97f7f51450dc4c
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
  Created wheel for markdown: filename=Markdown-2.2.0-cp36-none-any.whl size=136294 sha256=97ac210bae9414aa2a43632aaf85eac0ff1c6db180c0df28ac0f068ce280c8e0
  Stored in directory: /root/.cache/pip/wheels/b6/52/17/f0af18e3e0ec6fa60b361ffed15b4c3468f6f3bcdb87fbe079
Successfully built html5lib markdown
ERROR: tensorboard 2.3.0 has requirement markdown>=2.6.8, but you'll have markdown 2.2.0 which is incompatible.
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installatio

### O componente keras_adversarial funciona penas no Keras 2.1.2 e no tensorflow 1.2



In [3]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Flatten, Reshape
from keras.regularizers import L1L2
from keras_adversarial import AdversarialModel, simple_gan, gan_targets
from keras_adversarial import AdversarialOptimizerSimultaneous, normal_latent_sampling

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:461: FutureWarn

ImportError: ignored

*pip install --updgrade keras, volta a versão atual.*

In [5]:
(previsores_treinamento, _), (_,_) = mnist.load_data()
previsores_treinamento.shape

11493376/11490434 [==============================] - 0s 0us/step


(60000, 28, 28)

In [6]:
previsores_treinamento = previsores_treinamento.astype('float32') / 255
previsores_treinamento

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

# Criação das redes neurais

### Gerador

In [9]:
gerador = Sequential()

gerador.add(Dense(units = 500, 
                  input_dim = 100, 
                  activation = 'relu',
                  kernel_regularizer = L1L2(1e-5, 1e-5)))# 1e-5, 1e-5 definido na documentação 

gerador.add(Dense(units = 500, 
                  input_dim = 100, 
                  activation = 'relu',
                  kernel_regularizer = L1L2(1e-5, 1e-5)))

gerador.add(Dense(units = 784, 
                  activation = 'sigmoid', 
                  kernel_regularizer = L1L2(1e-5, 1e-5)))# 28 * 28

gerador.add(Reshape((28,28)))

### Discriminador

In [10]:
discriminador = Sequential()

discriminador.add(InputLayer(input_shape = (28,28))) # camada de entrada

discriminador.add(Flatten())

discriminador.add(Dense(units = 500, 
                        activation = 'relu', 
                        kernel_regularizer = L1L2(1e-5, 1e-5)))

discriminador.add(Dense(units = 500, 
                        activation = 'relu', 
                        kernel_regularizer = L1L2(1e-5, 1e-5)))

discriminador.add(Dense(units = 1, 
                        activation = 'sigmoid', 
                        kernel_regularizer = L1L2(1e-5, 1e-5)))

In [12]:
gan = simple_gan(gerador, discriminador, normal_latent_sampling((100,)))# valor igual ao input_dim do gerador

In [13]:
model = AdversarialModel(base_model = gan,
                         player_params = [gerador.trainable_weights,
                                          discriminador.trainable_weights])

In [15]:
model.adversarial_compile(adversarial_optimizer = AdversarialOptimizerSimultaneous,
                          player_optimizers = ['adam', 'adam'],
                          loss = 'binary_crossentropy')

In [19]:
model.fit(x = previsores_treinamento, 
          y = gan_targets(60000), epochs = 1, batch_size = 256)

AttributeError: ignored